# Cálculo de ETo (Piracicaba) pelo pacote PyETo

### Período: 2012-2021

In [1]:
import pandas as pd
import pyeto as pyeto
df = pd.read_csv('dados_PM.csv', sep = ';')
df

,date,latitude_graus,longitude_graus,altitude,julian_day,day,T_mean,RH_mean,U_z,T_max,T_min,precipitation,R_n,R_n (W/m²)
0,06/14/2022,-47.63,-22.7,546,165,14,16.90,66.27,0.43,25.7,8.1,0.000,5.546,NaN
1,06/15/2022,-47.63,-22.7,546,166,15,17.75,69.88,0.36,25.7,9.8,0.000,4.672,NaN
2,06/16/2022,-47.63,-22.7,546,167,16,19.90,69.81,0.29,30.3,9.5,0.000,5.937,NaN
3,06/17/2022,-47.63,-22.7,546,168,17,24.00,66.40,0.52,30.9,17.1,0.000,6.359,NaN
4,06/18/2022,-47.63,-22.7,546,169,18,19.05,83.05,0.41,25.8,12.3,1.778,3.084,NaN
5,06/19/2022,-47.63,-22.7,546,170,19,20.30,80.74,0.46,27.0,13.6,0.000,4.630,NaN
6,06/20/2022,-47.63,-22.7,546,171,20,17.05,86.45,0.26,22.4,11.7,0.000,1.510,NaN
7,06/21/2022,-47.63,-22.7,546,172,21,21.80,73.86,0.58,29.6,14.0,0.000,6.780,NaN
8,06/22/2022,-47.63,-22.7,546,173,22,21.45,66.75,0.57,30.8,12.1,0.000,6.380,NaN
9,06/23/2022,-47.63,-22.7,546,174,23,21.00,68.09,0.44,31.0,11.0,NaN,6.140,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             14 non-null     object 
 1   latitude_graus   14 non-null     float64
 2   longitude_graus  14 non-null     float64
 3   altitude         14 non-null     int64  
 4   julian_day       14 non-null     int64  
 5   day              14 non-null     int64  
 6   T_mean           14 non-null     float64
 7   RH_mean          14 non-null     float64
 8   U_z              14 non-null     float64
 9   T_max            14 non-null     float64
 10  T_min            14 non-null     float64
 11  precipitation    9 non-null      float64
 12  R_n              14 non-null     float64
 13  R_n (W/m²)       0 non-null      float64
dtypes: float64(10), int64(3), object(1)
memory usage: 1.7+ KB


In [5]:
#determinação da ETo
df['svp'] = df.apply(lambda x: pyeto.svp_from_t(x['T_mean']), axis = 1)
df['svp_tmin'] = df.apply(lambda x: pyeto.svp_from_t(x['T_min']), axis = 1)
df['svp_tmax'] = df.apply(lambda x: pyeto.svp_from_t(x['T_max']), axis = 1)
df['avp'] = df.apply(lambda x: pyeto.avp_from_rhmean(x['svp_tmin'], x['svp_tmax'], x['RH_mean']), axis=1)
df['delta_svp'] = df.apply(lambda x: pyeto.delta_svp(x['T_mean']), axis = 1)
df['atmos_pres'] = df.apply(lambda x: pyeto.atm_pressure(x['altitude']), axis = 1)
df['psy_const'] = df.apply(lambda x: pyeto.psy_const(x['atmos_pres']), axis = 1)
df['shf'] = 0
df['Tmean_K'] = df.apply(lambda x: pyeto.celsius2kelvin(x['T_mean']), axis=1)
df['ETo_PM'] = df.apply(lambda x: pyeto.fao56_penman_monteith(x['R_n'], x['Tmean_K'], x['U_z'], x['svp'], x['avp'], x['delta_svp'], x['psy_const'], x['shf']), axis=1)

In [4]:
df

,date,latitude_graus,longitude_graus,altitude,julian_day,day,T_mean,RH_mean,U_z,T_max,...,svp,svp_tmin,svp_tmax,avp,delta_svp,atmos_pres,psy_const,shf,Tmean_K,ETo_PM
0,06/14/2022,-47.63,-22.7,546,165,14,16.90,66.27,0.43,25.7,...,1.925484,1.080097,3.302286,1.452103,0.122113,95.010269,0.063182,0,290.05,1.625539
1,06/15/2022,-47.63,-22.7,546,166,15,17.75,69.88,0.36,25.7,...,2.031767,1.211602,3.302286,1.577153,0.127996,95.010269,0.063182,0,290.90,1.387425
2,06/16/2022,-47.63,-22.7,546,167,16,19.90,69.81,0.29,30.3,...,2.323846,1.187421,4.316625,1.921187,0.143959,95.010269,0.063182,0,293.05,1.740490
3,06/17/2022,-47.63,-22.7,546,168,17,24.00,66.40,0.52,30.9,...,2.983917,1.950043,4.467079,2.130484,0.179094,95.010269,0.063182,0,297.15,2.168421
4,06/18/2022,-47.63,-22.7,546,169,18,19.05,83.05,0.41,25.8,...,2.204257,1.430551,3.321903,1.973456,0.137457,95.010269,0.063182,0,292.20,0.913711
5,06/19/2022,-47.63,-22.7,546,170,19,20.30,80.74,0.46,27.0,...,2.382059,1.557578,3.565340,2.068122,0.147107,95.010269,0.063182,0,293.45,1.389262
6,06/20/2022,-47.63,-22.7,546,171,20,17.05,86.45,0.26,22.4,...,1.943878,1.375058,2.709082,1.765370,0.123134,95.010269,0.063182,0,290.20,0.442700
7,06/21/2022,-47.63,-22.7,546,172,21,21.80,73.86,0.58,29.6,...,2.611872,1.598605,4.146682,2.121734,0.159437,95.010269,0.063182,0,294.95,2.109287
8,06/22/2022,-47.63,-22.7,546,173,22,21.45,66.75,0.57,30.8,...,2.556585,1.411839,4.441691,1.953616,0.156485,95.010269,0.063182,0,294.60,2.042486
9,06/23/2022,-47.63,-22.7,546,174,23,21.00,68.09,0.44,31.0,...,2.487005,1.312714,4.492592,1.976417,0.152757,95.010269,0.063182,0,294.15,1.890513


In [6]:
df.to_excel('ETo_PM_predict.xlsx', index = False)